In [1]:
import numpy as np
import polars as pl
from matplotlib import pyplot as plt
import numpy as np 


todos: fill ratings (although they are not constant so probably just ffill), industry codes (same reasoning)



# Evaluating signals by quantiles

Guide by Barclays

1) Map Signals to alpha estimates, to make the signal comparable
2) Combine estimates into a composite score

## (1) Map to alpha estimates

* Divide through order statistics (big assumption: quantiles differences have always the same return implications)
* Create quintile/decile portfolios, plot how the Sharpe Ratios change as the signal value increases

Computing residual returns

* Partition bonds by (i) sector, (ii) rating category (Barclays just divides between A-level and B-level ratings) sectors are consolidated BCLASS level-3 classifications from BBG
* Systematic return: product of relative DTS * weighted average return of its sector-rating cell
    * Relative DTS: bond DTS - weighted average DTS of its sector-rating cell
* Residual: unadjusted return - systematic return

Dividing in quantiles

* Divide in quantiles, compute expected alpha
* You now get a structure of the function $\alpha|signal$
* Fit a function through it (if there is a no-information zone, zero it out
* Further standardization through z-scores? (see paper) 

In [2]:
bond_data = pl.read_parquet('../../data/bond_data_with_oas.pq')

bond_data.head(1)

,date,cusip,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,ind_num_17,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,CONVEXITY,CS_6M_DELTA,bond_value,BOND_VALUE,coupon,bond_type,…,NCOUPS,AMOUNT_OUTSTANDING,R_SP,R_MR,R_FR,N_SP,N_MR,N_FR,RATING_NUM,RATING_CAT,RATING_CLASS,T_DATE,T_Volume,T_DVolume,T_Spread,T_Yld_Pt,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,REMCOUPS,DEFAULTED,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE,DURATION,oas
i64,date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,…,f64,f64,str,str,str,f64,f64,f64,f64,str,str,date,str,str,str,f64,str,f64,f64,f64,f64,f64,date,f64,f64,f64,str,str,str,f64,str,str,str,str,str,f64,f64
0,2002-08-31,"""000336AE7""",-0.056089,-0.087082,-0.081158,-0.054689,-0.008212,-0.009612,-0.023343,8.0,0.043019,null,0.073689,100000.0,100000.0,97.693,0.97693,70.033333,4.585289,null,0.023939,3.597388,null,2.0,0.0,1.0,0.064316,0.032526,102.092,103.75346,26.445978,null,9.7693e6,1.02092e7,6.875,"""CDEB""",…,2.0,100000.0,"""BBB+""","""BAA2""","""BBB+""",8.0,9.0,8.0,8.0,"""BBB""","""0.IG""",2002-08-30,"""$119,000""","""$118,742""",null,6.918,"""7.369%""",97.7,97.7,97.7,1.0,44.0,2002-12-01,0.0,1.71,1.0,"""-.8212%""",null,"""-.8212%""",12.0,"""N""",null,null,null,null,4.59,0.039615


In [7]:
bond_data.select(['R_SP', 'N_SP']).unique().write_csv('ratings.csv')

In [3]:
bond_data.select(['R_SP', 'R_MR', 'R_FR', 'RATING_NUM', 'RATING_CAT', 'RATING_CLASS', 'rating']).null_count()

R_SP,R_MR,R_FR,RATING_NUM,RATING_CAT,RATING_CLASS,rating
u32,u32,u32,u32,u32,u32,u32
582,13612,84235,3464,22,6344,0


In [4]:
bond_data.select('RATING_CAT').unique()

RATING_CAT
str
"""BBB"""
"""A"""
"""BB"""
"""D"""
"""NR"""
…
"""C"""
"""AAA"""
"""CC"""


In [5]:
bond_data.group_by(['RATING_CAT']).n_unique()

RATING_CAT,,date,cusip,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,ind_num_17,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,CONVEXITY,CS_6M_DELTA,bond_value,BOND_VALUE,coupon,…,LAST_INTEREST_DATE,NCOUPS,AMOUNT_OUTSTANDING,R_SP,R_MR,R_FR,N_SP,N_MR,N_FR,RATING_NUM,RATING_CLASS,T_DATE,T_Volume,T_DVolume,T_Spread,T_Yld_Pt,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,REMCOUPS,DEFAULTED,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE,DURATION,oas
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""AA""",73248,242,2325,59166,58906,58906,59165,59588,59589,59325,8,72690,62875,72926,661,296,59826,59828,8285,72690,15,65116,62617,5114,25,2,1,68281,68731,56771,68731,68731,58559,64189,67333,442,…,1426,4,662,6,15,13,4,14,12,3,1,4130,46136,72967,561,43058,11396,1000,917,963,53,476,3440,306,470,17,25728,20098,23952,258,1,2,2,2,2,2422,65007
"""BBB""",544630,242,12256,398878,397561,397561,398871,401156,401161,399791,18,541124,474088,540648,4844,1542,318787,318778,10076,541098,18,465641,437358,31889,24,2,1,479149,502297,226337,502299,502299,435226,423399,448075,772,…,3064,4,4870,5,23,17,4,20,16,3,1,5041,126222,530978,1113,88084,15452,1479,1339,1441,88,561,5242,431,535,27,39540,35506,38281,299,1,48,4,3,50,5158,458156
"""AAA""",17555,242,467,14077,14033,14033,14077,14144,14144,14099,2,17429,14521,17434,217,108,14865,14865,5116,17429,11,15474,14802,1110,24,2,1,16397,16429,15296,16429,16429,13553,15355,16220,196,…,384,3,218,2,8,11,2,7,10,1,1,2355,13813,17540,429,14740,6837,738,675,728,31,342,1328,181,426,10,11838,7627,10503,222,1,1,1,1,1,1847,15520
"""A""",435449,242,10260,322518,321667,321664,322517,324608,324610,323715,11,433204,381310,432996,2649,1084,265288,265276,10078,433190,18,374518,352101,25742,25,2,1,387821,403059,201037,403059,403059,348604,343282,363608,746,…,3530,4,2652,5,20,17,4,19,16,3,1,5034,109617,426065,958,74982,16406,1333,1225,1300,114,603,6553,409,511,36,37767,33110,36405,354,1,15,3,3,15,4020,368942
"""C""",1094,225,234,756,765,765,739,836,849,813,10,1010,962,938,181,93,885,885,576,1029,17,961,914,211,23,1,2,1022,1022,959,1022,1022,917,927,1011,79,…,189,1,182,2,8,12,2,7,11,1,1,573,1063,1093,520,1046,878,640,454,591,11,187,223,62,398,3,985,440,816,42,1,94,4,3,89,467,936
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,22,6,20,5,5,5,5,8,8,8,8,22,15,20,18,15,11,11,22,22,9,4,4,4,5,2,1,7,7,7,7,7,7,11,7,18,…,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,11
"""CC""",2860,222,456,1926,1947,1946,1910,2016,2024,1991,8,2768,2493,2707,347,148,1953,1953,848,2798,18,2356,2181,422,23,1,2,2583,2583,2175,2583,2583,2202,2260,2522,111,…,297,2,348,2,11,18,2,10,16,1,1,1103,2596,2855,630,2636,1992,916,716,869,12,224,362,89,519,3,2498,996,1925,52,1,136,4,3,128,614,2330
"""D""",1786,201,592,939,970,965,921,1054,1083,1293,2,1584,1524,1480,448,166,1373,1373,807,1633,18,1583,1530,445,23,1,2,1674,1674,1485,1674,1674,1502,1504,1640,136,…,384,2,451,2,11,17,2,10,14,1,1,673,1700,1785,628,1692,1305,812,624,756,10,202,315,86,445,2,1258,479,1013,54,1,219,4,3,215,759,1533


Group by rating macro-categories

In [6]:
with pl.Config(tbl_rows=15): 
    print(bond_data.select('RATING_CAT').unique().to_series())

shape: (12,)
Series: 'RATING_CAT' [str]
[
	null
	"CC"
	"C"
	"CCC"
	"B"
	"AAA"
	"A"
	"BB"
	"BBB"
	"NR"
	"D"
	"AA"
]


In [7]:
rating_groups = {
    'A': ['A', 'AA', 'AAA'],
    'B': ['B', 'BB', 'BBB'],
}# C is all others

bond_data_with_ratings = bond_data.with_columns(
    rating_bucket=pl
        .when(pl.col('RATING_CAT').is_in(rating_groups['A'])).then(pl.lit('A'))
        .when(pl.col('RATING_CAT').is_in(rating_groups['B'])).then(pl.lit('B')).otherwise(pl.lit('C'))
)
bond_data = bond_data_with_ratings

Group by industry number

In [8]:
bond_data.select(['ind_num_17']).null_count()

ind_num_17
u32
344629


In [9]:
bond_data.group_by(['ind_num_17', 'date']).n_unique().min()

ind_num_17,date,,cusip,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,CONVEXITY,CS_6M_DELTA,bond_value,BOND_VALUE,coupon,bond_type,…,AMOUNT_OUTSTANDING,R_SP,R_MR,R_FR,N_SP,N_MR,N_FR,RATING_NUM,RATING_CAT,RATING_CLASS,T_DATE,T_Volume,T_DVolume,T_Spread,T_Yld_Pt,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,REMCOUPS,DEFAULTED,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE,DURATION,oas,rating_bucket
f64,date,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
1.0,2002-08-31,8,8,1,1,1,1,3,3,3,4,8,8,8,5,5,5,5,8,8,8,7,2,6,1,1,8,8,8,8,8,1,5,8,8,1,…,5,4,4,2,4,4,1,4,2,1,1,8,8,8,8,8,8,1,5,1,7,5,1,8,1,8,1,3,7,1,1,1,1,1,8,5,1


In [10]:
bond_data = bond_data.with_columns(
    industry_bucket=(pl.col('ind_num_17') % 3)
)
bond_data.select('industry_bucket')

industry_bucket
f64
null
null
null
1.0
1.0
…
1.0
1.0
1.0


In [11]:
bond_data.group_by(['date', 'industry_bucket', 'rating_bucket']).mean()

date,industry_bucket,rating_bucket,,cusip,exretn_t+1,exretnc_t+1,exretnc_dur_t+1,bond_ret_t+1,bond_ret,exretn,exretnc_dur,rating,cs,cs_6m_delta,bond_yield,bond_amount_out,offering_amt,bondprc,perc_par,tmt,duration,ind_num_17,BOND_RET,ILLIQ,var95,n_trades_month,size_ig,size_jk,BOND_YIELD,CS,BONDPRC,PRFULL,CONVEXITY,CS_6M_DELTA,bond_value,BOND_VALUE,…,NCOUPS,AMOUNT_OUTSTANDING,R_SP,R_MR,R_FR,N_SP,N_MR,N_FR,RATING_NUM,RATING_CAT,RATING_CLASS,T_DATE,T_Volume,T_DVolume,T_Spread,T_Yld_Pt,YIELD,PRICE_EOM,PRICE_LDM,PRICE_L5M,GAP,COUPMONTH,nextcoup,COUPAMT,COUPACC,MULTICOUPS,RET_EOM,RET_LDM,RET_L5M,REMCOUPS,DEFAULTED,DEFAULT_DATE,DEFAULT_TYPE,REINSTATED,REINSTATED_DATE,DURATION,oas
date,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,str,str,str,f64,f64,f64,f64,str,str,date,str,str,str,f64,str,f64,f64,f64,f64,f64,date,f64,f64,f64,str,str,str,f64,str,str,str,str,str,f64,f64
2011-02-28,1.0,"""C""",968650.0625,null,0.00739,0.009231,0.009282,0.00749,0.019096,0.018996,0.022523,17.53125,0.090609,-0.271098,0.104397,390990.6875,470344.21875,94.612329,0.946123,61.547917,3.533671,12.25,0.011437,1.575204,0.152514,10.551724,1.0,1.0,0.105644,0.090902,94.258819,97.0341,20.352334,-0.277056,3.9252e7,3.9202e7,…,2.0,390990.6875,null,null,null,17.633333,17.2,14.555556,17.633333,null,null,2011-02-26,null,null,null,9.785404,null,94.671875,95.126316,94.616129,1.0,48.866667,2011-04-16,0.0878125,2.857187,1.0,null,null,null,10.84375,null,null,null,null,null,3.534063,NaN
2010-12-31,null,"""A""",679773.191413,null,0.009444,0.014495,0.008101,0.009544,-0.013202,-0.013302,0.010168,5.751342,0.020895,-0.150439,0.04161,164538.841302,199953.919499,109.093078,1.090931,137.35653,7.069343,null,-0.007715,1.5296,0.058604,7.491139,0.001789,1.0,0.04226,0.019408,109.311567,110.699094,87.12674,-0.097208,1.9788e7,1.9354e7,…,2.486583,164538.841302,null,null,null,5.751351,6.092764,5.92,5.753131,null,null,2010-12-24,null,null,null,4.612397,null,127.00644,108.731481,109.095157,1.419238,109.411111,2011-04-01,0.724544,1.427406,1.037567,null,null,null,31.692308,null,null,null,null,null,7.040313,NaN
2004-06-30,0.0,"""A""",772660.976923,null,0.011827,0.003685,0.00511,0.012827,0.00131,0.00051,-0.00451,5.269231,0.011422,-0.039971,0.049054,495875.130769,480176.4,107.56534,1.075653,122.634615,6.27128,8.723077,0.006485,0.87725,0.034624,11.478632,1.0,1.0,0.048868,0.009991,107.216445,108.946226,71.570354,-0.065466,5.6970e7,5.4613e7,…,2.0,495875.130769,null,null,null,5.302326,5.555556,5.47561,5.292308,null,null,2004-06-28,null,null,null,4.895875,null,107.493846,106.822093,107.565179,1.053846,57.015748,2004-09-24,0.386923,1.723692,1.0,null,null,null,20.676923,null,null,null,null,null,6.270698,NaN
2017-11-30,0.0,"""B""",686334.870201,null,0.007847,0.005249,0.007741,0.008747,0.002544,0.001744,0.004513,10.25777,0.021491,-0.181165,0.043065,696968.659568,711397.246405,104.963336,1.049633,121.821755,6.806399,7.83181,0.001216,0.1334,0.049565,14.797422,1.0,1.0,0.042877,0.021587,105.116596,106.268702,86.733024,-0.151957,7.4202e7,7.3600e7,…,2.0,696929.567369,null,null,null,10.25777,10.728782,9.497093,10.25777,null,null,2017-11-29,null,null,null,3.980182,null,105.063071,104.448837,104.964607,1.003697,55.60735,2018-02-27,0.483711,1.233821,1.0,null,null,null,20.521024,null,null,null,null,null,6.80502,NaN
2013-05-31,2.0,"""C""",627721.008929,null,-0.036996,-0.017177,-0.023577,-0.036996,0.017478,0.017478,0.028561,17.535714,0.117089,-0.209613,0.125185,483443.767857,529543.053571,94.501893,0.945019,73.437798,4.146318,14.9375,0.008571,1.672305,0.116151,12.628571,1.0,1.0,0.133804,0.126006,95.899688,98.13363,28.067566,-0.205315,4.7481e7,4.4665e7,…,2.017857,483443.767857,null,null,null,17.775701,17.475248,17.809524,17.775701,null,null,2013-05-28,null,null,null,12.765204,null,95.902679,89.568966,94.503093,1.009259,48.719626,2013-08-27,0.902232,2.138839,1.0,null,null,null,12.714286,null,null,null,null,null,4.146518

Compute the metric

In [12]:
grouping_cols = ['date', 'industry_bucket', 'rating_bucket']

def calculate_avg_metric_over_peers(df: pl.DataFrame, col: str):
    bucket_means = df.select(grouping_cols + [col]).group_by(grouping_cols).agg(pl.col(col).drop_nans().mean().alias(f'{col}_bucket_mean'))
    
    return df.join(bucket_means, how='left', on=grouping_cols)

def calculate_quantile(df: pl.DataFrame, col: str, n_quantiles: int):
    new_col_name = f'{col}_bucket_quantile'
    mean_col_name = f'{col}_bucket_mean'
    
    bins = np.linspace(0,1,n_quantiles)
    labels = [f'q{i}' for i in range(n_quantiles+1)]
    
    bond_with_mean = calculate_avg_metric_over_peers(df, col)
    bond_with_quantiles = bond_with_mean.filter((pl.col(col).is_not_nan()) & (pl.col(mean_col_name).is_not_nan())).with_columns(
        (pl.col(col) - pl.col(mean_col_name)).qcut(bins, labels=labels).alias(new_col_name)       
    )
    
    return df.join(bond_with_quantiles, on=grouping_cols, how='left')

In [ ]:
calculate_quantile(bond_data, 'oas', 5).head()

In [40]:
calculate_avg_metric_over_peers(bond_data, 'oas').sort('date').head()

date,industry_bucket,rating_bucket,oas_bucket_mean
date,f64,str,f64
2002-08-31,null,"""A""",0.020703
2002-08-31,2.0,"""B""",0.06582
2002-08-31,2.0,"""A""",0.018893
2002-08-31,0.0,"""A""",0.014488
2002-08-31,2.0,"""C""",0.265647


In [59]:
bond_data.select('oas').to_series().drop_nans().qcut([0.1,0.9], labels=['one', 'two', 'three'])

oas
cat
"""two"""
"""two"""
"""two"""
"""two"""
"""two"""
…
"""two"""
"""two"""
"""two"""
